In [1]:
from src.Base.Evaluation.K_Fold_Evaluator import K_Fold_Evaluator_MAP
from src.Utils.ICM_preprocessing import *
from src.Utils.load_ICM import load_ICM
from src.Utils.load_URM import load_URM

URM_all = load_URM("../../../in/data_train.csv")
ICM_all = load_ICM("../../../in/data_ICM_title_abstract.csv")
from src.Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample

URMs_train = []
URMs_validation = []

for k in range(3):
    URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.80)
    URMs_train.append(URM_train)
    URMs_validation.append(URM_validation)

evaluator_validation = K_Fold_Evaluator_MAP(URMs_validation, cutoff_list=[10], verbose=False)

ICMs_combined = []
for URM in URMs_train:
    ICMs_combined.append(combine(ICM=ICM_all, URM=URM))

In [2]:
from src.Hybrid.GeneralizedMergedHybridRecommender import GeneralizedMergedHybridRecommender
from src.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Utils.confidence_scaling import linear_scaling_confidence
from src.Implicit.ImplicitALSRecommender import ImplicitALSRecommender
from src.SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython

from bayes_opt import BayesianOptimization

itemCBF_recommenders = []
itemCF_recommenders = []
userCF_recommenders = []
IALS_recommenders=[]
slimBPR_recommenders = []
recommenders = []

for index in range(len(URMs_train)):

    itemCF_recommenders.append(
        ItemKNNCFRecommender(
            URM_train=URMs_train[index],
            verbose=False
        )
    )

    itemCF_recommenders[index].fit(
        similarity="jaccard",
        topK=200,
        shrink=200,
        normalize=False
    )

    itemCBF_recommenders.append(
        ItemKNNCBFRecommender(
            URM_train=URMs_train[index],
            ICM_train=ICM_all,
            verbose=False
        )
    )

    itemCBF_recommenders[index].fit(
        similarity="jaccard",
        topK=200,
        shrink=10,
        normalize=False
    )

    userCF_recommenders.append(
        UserKNNCFRecommender(
            URM_train=URMs_train[index],
            verbose=False
        )
    )

    userCF_recommenders[index].fit(
        similarity="cosine",topK=524,shrink=52, normalize=False, feature_weighting="TF-IDF"
    )

    IALS_recommenders.append(
        ImplicitALSRecommender(
            URM_train=URMs_train[index],
            verbose=False
        )
    )

    IALS_recommenders[index].fit(
        factors=int(188.56989379722654),
        regularization=0.01,
        use_gpu=False,
        iterations=int(10),
        num_threads=4,
        confidence_scaling=linear_scaling_confidence,
        **{"alpha": 50}
    )

    slimBPR_recommenders.append(
        SLIM_BPR_Cython(
                URM_train=URMs_train[index],
            verbose=False
        )
    )

    slimBPR_recommenders[index].fit(
        epochs=98.33218638454095,
        positive_threshold_BPR=None,
        train_with_sparse_weights=True,
        symmetric=False,
        random_seed=None,
        batch_size=548.6615810947892,
        lambda_i=0.0039954585909894764,
        lambda_j=0.008139187451093313,
        learning_rate=1e-4,
        topK=int(591.8108481752299),
        sgd_mode='adagrad',
        # gamma=,
        # beta_1=,
        # beta_2=,
    )

    recommenders.append(
        GeneralizedMergedHybridRecommender(
        URM_train=URMs_train[index],
        recommenders=[
            itemCF_recommenders[index],
            itemCBF_recommenders[index],
            userCF_recommenders[index],
            IALS_recommenders[index],
            slimBPR_recommenders[index]
        ],
        verbose=False
    )
    )

Similarity column 25975 ( 100 % ), 6275.81 column/sec, elapsed time 0.07 min
Similarity column 25975 ( 100 % ), 4004.58 column/sec, elapsed time 0.11 min
Similarity column 7947 ( 100 % ), 17337.26 column/sec, elapsed time 0.01 min
SLIM_BPR_Recommender: Epoch 1 of 98.33218638454095. Elapsed time 0.05 sec
SLIM_BPR_Recommender: Epoch 2 of 98.33218638454095. Elapsed time 0.15 sec
SLIM_BPR_Recommender: Epoch 3 of 98.33218638454095. Elapsed time 0.38 sec
SLIM_BPR_Recommender: Epoch 4 of 98.33218638454095. Elapsed time 0.64 sec
SLIM_BPR_Recommender: Epoch 5 of 98.33218638454095. Elapsed time 0.92 sec
SLIM_BPR_Recommender: Epoch 6 of 98.33218638454095. Elapsed time 1.24 sec
SLIM_BPR_Recommender: Epoch 7 of 98.33218638454095. Elapsed time 1.69 sec
SLIM_BPR_Recommender: Epoch 8 of 98.33218638454095. Elapsed time 2.18 sec
SLIM_BPR_Recommender: Epoch 9 of 98.33218638454095. Elapsed time 2.79 sec
SLIM_BPR_Recommender: Epoch 10 of 98.33218638454095. Elapsed time 3.42 sec
SLIM_BPR_Recommender: Epoch 

In [3]:
import numpy as np

results = []

for weight1 in np.linspace(start=0.8,stop=1.0,num=10):
    for index in range(len(URMs_train)):

        recommenders[index].fit(
            alphas=[
                weight1,
                1-weight1
                ]
            )

    result = evaluator_validation.evaluateRecommender(recommenders)

    results.append(
        {
            'target': sum(result) / len(result),
            'params': {
            'alphas': [
                        weight1,
                        1-weight1
                    ]
            }
        }
    )

    print(results[-1])

{'target': 0.05750050946813428, 'params': {'alphas': [0.8, 0.19999999999999996]}}
{'target': 0.059111933079542676, 'params': {'alphas': [0.8222222222222223, 0.1777777777777777]}}
{'target': 0.0608807566397123, 'params': {'alphas': [0.8444444444444444, 0.15555555555555556]}}
{'target': 0.06280259309880337, 'params': {'alphas': [0.8666666666666667, 0.1333333333333333]}}
{'target': 0.06463309964148665, 'params': {'alphas': [0.888888888888889, 0.11111111111111105]}}
{'target': 0.06581120320702115, 'params': {'alphas': [0.9111111111111111, 0.0888888888888889]}}
{'target': 0.06618301281086286, 'params': {'alphas': [0.9333333333333333, 0.06666666666666665]}}
{'target': 0.06493520334949944, 'params': {'alphas': [0.9555555555555555, 0.04444444444444451]}}
{'target': 0.060867630937454474, 'params': {'alphas': [0.9777777777777777, 0.022222222222222254]}}
{'target': 0.05151060107598171, 'params': {'alphas': [1.0, 0.0]}}


In [4]:
maxParams =max(results, key= lambda x: x['target'])

print('The best configuration is params = {}, with MAP {}'.format(maxParams['params'], maxParams['target']))

The best configuration is params = {'alphas': [0.9333333333333333, 0.06666666666666665]}, with MAP 0.06618301281086286


In [8]:
import numpy as np

results = []

for weight2 in np.linspace(start=0.0,stop=0.01,num=10):
    for index in range(len(URMs_train)):

        recommenders[index].fit(
            alphas=
            maxParams['params']['alphas']
            +[weight2]
            )

    result = evaluator_validation.evaluateRecommender(recommenders)

    results.append(
        {
            'target': sum(result) / len(result),
            'params': {
            'alphas': maxParams['params']['alphas']+[weight2]

            }
        }
    )

    print(results[-1])

{'target': 0.06618301281086286, 'params': {'alphas': [0.9333333333333333, 0.06666666666666665, 0.0]}}
{'target': 0.06509584445731659, 'params': {'alphas': [0.9333333333333333, 0.06666666666666665, 0.0011111111111111111]}}
{'target': 0.06200652802152649, 'params': {'alphas': [0.9333333333333333, 0.06666666666666665, 0.0022222222222222222]}}
{'target': 0.060770147477133364, 'params': {'alphas': [0.9333333333333333, 0.06666666666666665, 0.003333333333333333]}}
{'target': 0.059802689311419265, 'params': {'alphas': [0.9333333333333333, 0.06666666666666665, 0.0044444444444444444]}}
{'target': 0.05924113486871565, 'params': {'alphas': [0.9333333333333333, 0.06666666666666665, 0.005555555555555556]}}
{'target': 0.05880996201370592, 'params': {'alphas': [0.9333333333333333, 0.06666666666666665, 0.006666666666666666]}}
{'target': 0.05867892639996963, 'params': {'alphas': [0.9333333333333333, 0.06666666666666665, 0.0077777777777777776]}}
{'target': 0.05841379228845933, 'params': {'alphas': [0.933

In [ ]:
import numpy as np

results = []

for weight1 in np.linspace(start=0,stop=1,num=5):
    for weight2 in np.linspace(start=0,stop=1,num=5):
        for weight3 in np.linspace(start=0, stop=1, num=5):
            for weight4 in np.linspace(start=0, stop=1, num=5):
                for weight5 in np.linspace(start=0,stop=1,num=5):
                    for index in range(len(URMs_train)):

                        recommenders[index].fit(
                            alphas=[
                                weight1,
                                weight2,
                                weight3,
                                weight4,
                                weight5
                            ]
                        )

                    result = evaluator_validation.evaluateRecommender(recommenders)

                    results.append(
                        {
                            'target': sum(result) / len(result),
                            'params': {
                                'alphas': [
                                            weight1,
                                            weight2,
                                            weight3,
                                            weight4,
                                            weight5
                                        ]
                            }
                        }
                    )

                    print(results[-1])

import json

with open("logs/"+ recommenders[0].RECOMMENDER_NAME+"_logs.json", 'w') as json_file:
    json.dump(max(results, key= lambda x: x['target']), json_file)

